In [2]:
# python libraries
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np

# pydrake imports
from pydrake.all import MathematicalProgram, Solve
import pydrake.symbolic as sym

# SOS and Positive Semi-Definite functions
In this exercise we will empirically see that not all positive semi-definite functions have a sum-of-squares representation. We consider the following three polynomial functions for this purpose:

1. $f_1(x, y) = x^4y^2 + x^2y^4 + 1 - 3x^2y^2$
2. $f_2(x, y) = x^4 + 3y^4 - 2xy^3$
3. $f_3(x, y, z; \lambda) = z^6 + x^2y^2(x^2 + y^2 - 3\lambda z^2)$, where $0 < \lambda \le 1$.

In [ ]:
def f1(x):
    return x[0] ** 4 * x[1] ** 2 + x[0] ** 2 * x[1] ** 4 + 1 - 3 * x[0] ** 2 * x[1] ** 2


def f2(x):
    return x[0] ** 4 + 3 * x[1] ** 4 - 2 * x[0] * x[1] ** 3


def f3(x, a):
    return x[2] ** 6 + x[0] ** 2 * x[1] ** 2 * (
        x[0] ** 2 + x[1] ** 2 - 3 * a * x[2] ** 2
    )

**First, let us check whether the above functions admit an SOS representation.**

This can be verified using a single SOS constraint on the functions. In case of $f_3$, think about what one particular value of $\lambda$ is enough to search for an SOS representation for any $0 < \lambda \le 1$.

In [ ]:
# code for checking SOS polynomials goes below

Write the result in the next cell.


In [ ]:
is_function_sos = [False, False, False]  # MODIFY HERE

**Now, let us find whether the above polynomial functions are positive semi-definite, that is, $f(\mathbf{x}) \ge 0$.**
You can try plotting the functions for a certain range of $\mathbf{x}$ values, or find the minimum of $f(\mathbf{x})$ by obtaining the roots of $\nabla f(\mathbf{x}) = \mathbf{0}$.

If you choose to verify by plotting the functions for some interval of $\mathbf{x}$ values (say $\mathbf{x} \in [-1, 1]^2$), reason for yourself why the chosen set is sufficient for verification. Also note that it becomes difficult to visualize functions as number of variables increases beyond two.
**Note**: We assume that the the discreteness used in plotting is sufficient to capture the bumps in the function values (the plots have enough resolution that we do not skip any negative value in our plots).

In the next cell, write down which functions are positive semi-definite.

In [ ]:
is_function_psd = [False, False, False]  # MODIFY HERE

Following is an example code for plotting a 2D function.

In [ ]:
# grid resolution and range
grid_res = 100
x_lims = (0, 1)
y_lims = (0, 1)
func = f2

# Get function values over a meshgrid
x = np.linspace(x_lims[0], x_lims[1], num=grid_res)
y = np.linspace(y_lims[0], y_lims[1], num=grid_res)
X, Y = np.meshgrid(x, y)

Z = np.zeros((len(x), len(y)))
for i in range(len(x)):
    for j in range(len(y)):
        Z[i, j] = func([x[i], y[j]])

# add a figure
fig = plt.figure(figsize=(12, 6))

# subplot 1: 2D heatmap
ax1 = fig.add_subplot(1, 2, 1)
c = ax1.pcolormesh(x, y, Z, cmap="plasma", vmin=np.min(Z), vmax=np.max(Z))
ax1.set_title(f"$\min \; f_2(x, y) = {np.min(Z)}$")
ax1.axis([x.min(), x.max(), y.min(), y.max()])
ax1.set_aspect("equal")
ax1.set_xlabel("x")
ax1.set_ylabel("y")
fig.colorbar(c, ax=ax1)

# subplot 2: 3D surface plot
ax2 = fig.add_subplot(1, 2, 2, projection="3d")
ax2.plot_surface(
    X,
    Y,
    Z,
    rstride=1,
    cstride=1,
    facecolors=cm.plasma(Z / np.amax(Z)),
    linewidth=0,
    antialiased=False,
    shade=False,
    alpha=0.3,
)
ax2.set_xlabel("x")
ax2.set_ylabel("y")
ax2.set_zlabel("z")
ax2.set_title(f"$f_2(x, y)$")

plt.show()

Thus, we see that a function may be positive semi-definite functions even if it does not admit a SOS representation.

**Fun Exercise: Can we use SOS programming to find lower bounds for these polynomials?**
An alternative condition for a polynomial to be positive semi-definite is finding a lower bound larger than or equal to $0$, i.e., there exists $\gamma$ such that $f(x) -\gamma \geq 0$ and $\gamma \geq 0$.

A **necessary** condition for these conditions is that we can find $\gamma \geq 0$ such that $f(x) - \gamma$ is SOS. The cell block below implements such a SOS programming problem to search for maximum lower bound $\gamma$. Run the cell blocks to see whether the lower bound can be found for each of these polynomials and how tight the lower bounds are (if found successfully).

In [ ]:
def check_lb_sos(f, num_var=2):
    prog = MathematicalProgram()
    x = prog.NewIndeterminates(num_var, "x")
    c = prog.NewContinuousVariables(1)
    func = sym.Polynomial(f(x))
    prog.AddSosConstraint(func - c[0])

    prog.AddCost(-c[0])
    result = Solve(prog)
    return result.is_success(), result.GetSolution(c)

In [ ]:
success_1, c_1 = check_lb_sos(f1, num_var=2)
success_2, c_2 = check_lb_sos(f2, num_var=2)
success_3, c_3 = check_lb_sos(lambda x: f3(x, 1), num_var=3)
print("Lower bound found for f1: ", success_1)
print("Lower bound found for f2: ", c_2[0])
print("Lower bound found for f3: ", success_3)

## Autograding
You can check your work by runnig the next cell. Only your responses for `is_function_sos` and `is_function_psd` are graded.

In [ ]:
from underactuated.exercises.lyapunov.test_sos_and_psd import TestSOSandPSD
from underactuated.exercises.grader import Grader

Grader.grade_output([TestSOSandPSD], [locals()], "results.json")
Grader.print_test_results("results.json")